In [1]:
import pandas as pd
import dash
from dash import dash_table
import dash_daq as daq
from dash import html
from dash.dependencies import Input, Output


def zscore_filtration(df, price_zscore_val=3, quantity_zscore_val=3, paid_zscore_val=3):
    col = "unit price"
    df["ZSCORE_{} outlier".format(col)] = df['ZSCORE_{}'.format(col)] > price_zscore_val

    col = "requested quantity"
    df["ZSCORE_{} outlier".format(col)] = df['ZSCORE_{}'.format(col)] > quantity_zscore_val

    col = "Paid"
    df["ZSCORE_{} outlier".format(col)] = df['ZSCORE_{}'.format(col)] > paid_zscore_val

    return df


def get_std_label(x):
    if (x["ZSCORE_unit price outlier"] or x["ZSCORE_requested quantity outlier"]) and x["ZSCORE_Paid outlier"]:
        return 1
    else:
        return 0


df = pd.read_csv('Results/all_Medgulf_Statistical_dataframe.csv')
df = zscore_filtration(df)
df["Statistical_Label"] = df.apply(lambda x: get_std_label(x), axis=1)

PAGE_SIZE = 200

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div([
    html.Div(
        [
        html.Div([
            html.P("price_zscore_slider: "),
            daq.Slider(id='price_zscore_slider', min=0, max=4, value=3, handleLabel={"showCurrentValue": True, "label": "VALUE"}, step=0.2)],
            style={'display': 'inline-block', 'padding': '10px'}),

        html.Div([
            html.P("quantity_zscore_slider: "),
            daq.Slider(id='quantity_zscore_slider', min=0, max=4, value=3, handleLabel={"showCurrentValue": True, "label": "VALUE"}, step=0.2)],
            style={'display': 'inline-block', 'padding': '10px'}),

        html.Div([
            html.P("paid_zscore_slider: "),
            daq.Slider(id='paid_zscore_slider', min=0, max=4, value=3, handleLabel={"showCurrentValue": True, "label": "VALUE"}, step=0.2)],
            style={'display': 'inline-block', 'padding': '10px'})
        ],
        style={'display': 'flex', 'justify-content': "space-evenly"}),

    dash_table.DataTable(
        id='datatable-paging',
        columns=[
            {"name": i, "id": i, "hideable": "last"} for i in df.columns  # sorted(df.columns)
        ],
        editable=True,
        column_selectable="single",
        row_selectable="multi",
        row_deletable=True,
        selected_columns=[],
        selected_rows=[],
        page_current=0,
        page_size=PAGE_SIZE,
        page_action='custom',
        sort_action='custom',
        sort_mode='multi',
        sort_by=[],
        hidden_columns=[],
        filter_action='custom',
        filter_query='',
        css=[
            {"selector": ".column-header--delete svg", "rule": 'display: "none"'},
            {"selector": ".column-header--delete::before", "rule": 'content: "X"'}
        ]
    )
])

operators = [['ge ', '>='],
             ['le ', '<='],
             ['lt ', '<'],
             ['gt ', '>'],
             ['ne ', '!='],
             ['eq ', '='],
             ['contains '],
             ['datestartswith ']]


def split_filter_part(filter_part):
    for operator_type in operators:
        for operator in operator_type:
            if operator in filter_part:
                name_part, value_part = filter_part.split(operator, 1)
                name = name_part[name_part.find('{') + 1: name_part.rfind('}')]

                value_part = value_part.strip()
                v0 = value_part[0]
                if (v0 == value_part[-1] and v0 in ("'", '"', '`')):
                    value = value_part[1: -1].replace('\\' + v0, v0)
                else:
                    try:
                        value = float(value_part)
                    except ValueError:
                        value = value_part
                return name, operator_type[0].strip(), value
    return [None] * 3


@app.callback(
    Output('datatable-paging', 'data'),
    [Input('datatable-paging', "filter_query"),
     Input('datatable-paging', 'page_current'),
     Input('datatable-paging', 'page_size'),
     Input('datatable-paging', 'sort_by'),
     Input('price_zscore_slider', "value"),
     Input('quantity_zscore_slider', "value"),
     Input('paid_zscore_slider', "value")])
def update_table(filter, page_current, page_size, sort_by, price_zscore_val, quantity_zscore_val, paid_zscore_val):
    dff = df.copy()

    # Filter
    filtering_expressions = filter.split(' && ')
    for filter_part in filtering_expressions:
        col_name, operator, filter_value = split_filter_part(filter_part)

        if operator in ('eq', 'ne', 'lt', 'le', 'gt', 'ge'):
            dff = dff.loc[getattr(dff[col_name], operator)(filter_value)]
        elif operator == 'contains':
            dff = dff.loc[dff[col_name].str.contains(filter_value)]
        elif operator == 'datestartswith':
            dff = dff.loc[dff[col_name].str.startswith(filter_value)]

    dff = zscore_filtration(dff, price_zscore_val, quantity_zscore_val, paid_zscore_val)

    # Sort if necessary
    if len(sort_by):
        dff = df.sort_values(
            [col['column_id'] for col in sort_by],
            ascending=[
                col['direction'] == 'asc'
                for col in sort_by
            ],
            inplace=False
        )

    return dff.iloc[
           page_current * page_size:(page_current + 1) * page_size
           ].to_dict('records')


if __name__ == '__main__':
    app.run_server(debug=True)

ModuleNotFoundError: ignored